## Classification of the MNIST data using CNNs

The MNIST Dataset has 28 x 28 black and white images of digits from 0 to 9. It is one of the most common datasets for starting up with deep learning. It comes in built in the keras package.

This notebook will walk you through the developing a classification model for the dataset using a <b>Convolutional Neural Network</b> with the help of the <b>Model API</b> from Keras.

### Importing required libraries

In [1]:
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt

from keras.layers import Input, Flatten, Dense, Dropout, MaxPooling2D
#from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import Conv2D

from keras.activations import *
from keras.optimizers import *
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Model, Sequential

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Loading the dataset

MNIST comes as a part of the keras datasets. It contains 60,000 training images while 10,000 test images

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Display one image?

In [3]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

### Data Preprocessing

The labels refer to the expected classification output for a given image. They are converted from singular to one-hot encoded values from 0 to 9. 

That is, if a given image corresponds to 5, its encoding will be [0,0,0,0,0,1,0,0,0,0]

- Since we are using a convolutional network we do not need to flatten the array into 1D.
- rescale pixel values between 0 and 1\n
- Input type should be float\n
- There are 10 classes so in order to compute the cross entropy loss function we need to one-hot encoded vectors.

In [4]:
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_test = (X_test.astype(np.float32) - 127.5)/127.5

X_train = X_train[:,:,:,np.newaxis]
X_test = X_test[:,:,:,np.newaxis]

X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [5]:
num_classes = 10

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
#y_test = to_categorical(y_test, num_classes)

### Creating a Keras Model

Construct a model using the sequential API with the following instructions:

- Inputs are normalized using BatchNormaliation followed by a Dropout layer with a rate of 0.3
- Then add a 2D convolutional layer with a kernel of 3x3 with 32 filters
- Activate the output using an Activation layer
- Output from the convolutional layer goes through a MaxPooling layer
- Add another 2D convolutional layer but ensure that the shape remains 'same' as previous layer having 64 filters
- Output from the convolutional layer goes through a MaxPooling layer
- Then Flatten the output and add a Dropout layer with a rate of 0.3
- Connect the output to a Dense layer containing 100 neurons followed by a BatchNormalization 
- Add 'relu' activation function
- and finally connect to the output layer with an softmax activation function

Print the model summary to see the network

In [6]:
In = Input(shape=(28,28,1))
conv = Conv2D(4, kernel_size=5, strides=1, padding="valid",activation='relu')
pool = MaxPooling2D(pool_size=2,strides=2, padding="same")
fc = Dense(num_classes, activation='softmax')

fmap1 = conv(In)
fmap2 = pool(fmap1)
dense = Flatten()(fmap2)
y = Dense(num_classes, activation='softmax')(dense)
# x = BatchNormalization()(x)

# x = Conv2D(64, kernel_size=3, strides=2, padding="same",activation='relu')(x)
# x = MaxPooling2D(pool_size=2,strides=2, padding="same")(x)
# x = BatchNormalization()(x)

# x = Flatten()(x)
# x = Dense(64, activation='relu')(x)
# y = Dense(num_classes, activation='softmax')(x)
                                                                                    

"""
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu")
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
"""                                                                                

model = Model(inputs=In,outputs=y)                                                                                   
                                                                                   
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 4)         104       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 4)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5770      
Total params: 5,874
Trainable params: 5,874
Non-trainable params: 0
_________________________________________________________________


The final model is compiled using an optimizer, a loss function and a metric for performance improvement. 
- The loss function is used to depict how far is the current model from the ideal answer
- The optimizer refers to the method that will be used to minimize the loss
- The metrics correspond to how we want to measure the performance of the network

In [7]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=['accuracy'])

### Training and testing

- Training is done using the function fit(). We train out network for 5 epochs.
- Testing is done using the predict() function. We can also use evaluate() but since we want to later generate a classificiation report, we will use the former


In [8]:
model.fit(X_train,y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 21s 351us/step - loss: 0.4212 - acc: 0.8795
Epoch 2/5
60000/60000 [==============================] - 19s 319us/step - loss: 0.1582 - acc: 0.9541
Epoch 3/5
60000/60000 [==============================] - 18s 296us/step - loss: 0.1222 - acc: 0.9645
Epoch 4/5
60000/60000 [==============================] - 18s 299us/step - loss: 0.1037 - acc: 0.9700
Epoch 5/5
60000/60000 [==============================] - 18s 297us/step - loss: 0.0928 - acc: 0.9725


In [9]:
y_check = model.predict(X_test)

y_pred = np.array([np.argmax(y_check[j]) for j in range(len(y_check))])
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [10]:
print(confusion_matrix(y_test, y_pred))

[[ 971    0    1    0    0    1    3    1    2    1]
 [   0 1125    4    0    0    1    3    0    2    0]
 [   2    3 1014    3    1    0    1    2    6    0]
 [   1    0    4  993    0    3    0    4    3    2]
 [   1    0    4    0  967    0    1    1    2    6]
 [   1    1    0    5    0  872    3    2    6    2]
 [   5    3    2    0    2    1  943    0    2    0]
 [   0    4   13    2    0    0    0  998    3    8]
 [   7    2    5    3    1    0    2    6  944    4]
 [   6    6    1    1    6    2    0   11    3  973]]


In [11]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.98      0.99      0.98       980
          1       0.98      0.99      0.99      1135
          2       0.97      0.98      0.97      1032
          3       0.99      0.98      0.98      1010
          4       0.99      0.98      0.99       982
          5       0.99      0.98      0.98       892
          6       0.99      0.98      0.99       958
          7       0.97      0.97      0.97      1028
          8       0.97      0.97      0.97       974
          9       0.98      0.96      0.97      1009

avg / total       0.98      0.98      0.98     10000



In [10]:
#conv.get_weights

In [11]:
x = []
for layer in model.layers: 
    x.append(layer.get_weights())
    #print(layer.get_weights())
    #print("\n\n\n\n")
    
wmap1 = np.array(x[1][0])
bmap1 = np.array(x[1][1])

wmap2 = np.array(x[4][0])
bmap2 = np.array(x[4][1])

#layer.get_config(), 

In [12]:
# f1 = open("wmap1.txt","w")
# f = wmap1.T.flatten('C')
# for i in range(11520/5):
#     try:
#         x =  str(f[i])+", "+str(f[i+1])+", "+str(f[i+2])+", "+str(f[i+3])+", "+str(f[i+4])+", \n"
#         f1.write(x)
#     except:
#         break
# f1.close()

In [13]:
# wmap1 = np.array(x[1][0])
# wmap2 = np.array(x[4][0])
# #writer = wmap1.flatten()
# #wmap1
wmap1.T

array([[[[ 0.188182  ,  0.07620052, -0.14894305, -0.39629802,
          -0.3198553 ],
         [ 0.25531337,  0.0342965 , -0.2747088 , -0.28007376,
          -0.2903393 ],
         [ 0.3933249 ,  0.34005094, -0.00532881, -0.2629006 ,
          -0.51142466],
         [ 0.45010862,  0.44546333,  0.15801878, -0.2590417 ,
          -0.51174414],
         [ 0.06208478,  0.5496921 ,  0.42084926,  0.39437827,
          -0.28109783]]],


       [[[ 0.05863997,  0.29105607,  0.46185175,  0.42564487,
           0.1958066 ],
         [ 0.25320193,  0.4064024 ,  0.4276098 ,  0.15823913,
           0.04433811],
         [ 0.38967368,  0.09549404,  0.0308607 , -0.27944162,
          -0.3492783 ],
         [ 0.19999553, -0.14460343, -0.07421932, -0.28957462,
          -0.3618506 ],
         [-0.1664491 , -0.21875365, -0.3681912 , -0.43953162,
          -0.49415025]]],


       [[[-0.36640587, -0.34695625, -0.47125834, -0.2334613 ,
          -0.0904766 ],
         [-0.5578055 , -0.15011507, -0.2144709

In [72]:
wmap = np.zeros((200,1))
dx = 0
for i in range(5):
    for j in range(5):
        for k in range(8):
            wmap[dx] = wmap1[i][j][0][k]
            dx += 1
wmap.T

array([[ 1.16742313e-01,  5.55352792e-02, -7.61901140e-02,
         5.39284408e-01, -2.09182113e-01, -1.78037643e-01,
         3.90237421e-01, -2.15333253e-01,  1.54524758e-01,
         7.55098881e-05, -1.38261288e-01,  3.63211483e-01,
        -2.87931636e-02, -1.39477476e-01,  6.52614057e-01,
        -3.22264999e-01,  2.29107872e-01, -9.85072404e-02,
        -1.35328501e-01,  2.28702769e-01, -2.62545258e-01,
        -1.42032087e-01,  6.27099693e-01, -2.08277598e-01,
         5.27418740e-02,  6.07124828e-02, -1.14080630e-01,
        -3.47269386e-01, -1.37009859e-01,  3.95289570e-01,
         6.24461353e-01, -1.60556898e-01, -3.19887698e-02,
        -1.52072012e-01,  1.43811330e-01, -4.33153123e-01,
        -1.65634707e-01,  3.77474278e-01,  5.36893547e-01,
        -2.01450378e-01,  1.82521775e-01,  5.54543138e-02,
        -1.26140311e-01,  5.00761449e-01, -2.15790182e-01,
        -8.37143883e-02,  2.80732781e-01, -3.14082891e-01,
         1.02859356e-01, -7.25525692e-02, -6.92386776e-0

In [60]:
#X_train[99].reshape((28,28,1))
#== wmap1.T.flatten(order='C')

In [14]:
#wmap1 = np.array(x[1][0])
writer = wmap1.T.flatten(order='C')
wmap1_base = 0x20000C54
f=open("init2.ini", "w")
for entry in writer:
    str1="_WFLOAT ("+str(wmap1_base)+","+str(entry)+") \n"
    f.write(str1)
    wmap1_base += 4
f.close()

# wmap = np.zeros((11520,1))
# dx = 0
# for i in range(1152):
#     for j in range(10):
#         #print i,j, dx
#         wmap[dx] = wmap2[i][j]
#         dx += 1
# wmap2 = wmap.flatten(order='C')

In [15]:
wmap2_base = 0x20000F74
#wmap2 = np.array(x[4][0])
writer = wmap2.T.flatten(order='C')
f=open("init2.ini", "a")
for entry in writer:
    str1="_WFLOAT ("+str(wmap2_base)+","+str(entry)+") \n"
    f.write(str1)
    wmap2_base += 4
f.close()

In [47]:
input_base = 0x20000014
input_map = X_train[455].T
writer = input_map.flatten(order='C')
f=open("init2.ini", "a")
idx=0
for entry in writer:
    str1="_WFLOAT ("+str(input_base)+","+str(entry)+") \n"
    f.write(str1)
    #print str1
    input_base += 4
    idx+=1
f.close()
idx

784

In [17]:
bmap1 = np.array(x[1][1])
#print bmap1.shape
writer = bmap1.flatten(order='C')
bmap1_base = 0x2000C374
f=open("init2.ini", "a")
for entry in writer:
    str1="_WFLOAT ("+str(bmap1_base)+","+str(entry)+") \n"
    f.write(str1)
    bmap1_base += 4
f.close()

In [18]:
bmap2 = np.array(x[4][1])
#print bmap2.shape
writer = bmap2.flatten(order='C')
bmap2_base = 0x2000C394 #020
f=open("init2.ini", "a")
for entry in writer:
    str1="_WFLOAT ("+str(bmap2_base)+","+str(entry)+") \n"
    f.write(str1)
    bmap2_base += 4
f.close()

In [50]:
y_train[455], model.predict(X_train[455][np.newaxis,:,:,:])

(array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([[1.0310464e-06, 9.9448830e-01, 6.5358420e-04, 7.2428084e-05,
         2.0706819e-04, 7.8716447e-08, 8.8869126e-07, 4.3997401e-03,
         1.4227125e-04, 3.4638411e-05]], dtype=float32))

In [49]:
for i in input_map.flatten():
    print str(i)+",",
    

-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0

In [35]:
mo = 255
np.argmax(y_train[mo])

3

In [36]:
model.predict(X_train[mo][np.newaxis,:,:,:])

array([[8.9544105e-10, 1.2597758e-07, 1.5826465e-05, 9.9981433e-01,
        3.3035694e-10, 5.5585137e-05, 1.8169173e-10, 3.3826855e-10,
        1.1387560e-04, 2.5390958e-07]], dtype=float32)

In [37]:
p = X_train[200].flatten('C')

f1 = open("image.txt","w")
for i in range(len(p)-1):
    x = str(p[i])+","
    f1.write(x)
x = str(p[i])
f1.write(x)
f1.close()

In [ ]:
len()

In [247]:
y_train[99]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [188]:
hex(0x1FF00000 + 128*1024*1024)

'0x27f00000'